In [103]:
import os
import sys
from xml.sax import parse
from xml.sax.saxutils import XMLGenerator
import pandas as pd

class CycleFile(object):

    def __init__(self, filename):
        self.basename, self.ext = os.path.splitext(filename)
        self.index = 0
        self.open_next_file()

    def open_next_file(self):
        self.index += 1
        self.file = open(self.name(), 'wb')

    def name(self):
        return '%s%s%s' % (self.basename, self.index, self.ext)

    def cycle(self):
        self.file.close()
        self.open_next_file()

    def write(self, str):
        self.file.write(str)

    def close(self):
        self.file.close()

class XMLBreaker(XMLGenerator):
    
    def __init__(self, break_into=None, break_after=1000, out=None, *args, **kwargs):
        XMLGenerator.__init__(self, out, encoding='utf-8', *args, **kwargs)
        self.out_file = out
        self.break_into = break_into
        self.break_after = break_after
        self.context = []
        self.count = 0
        self.file_names = [self.out_file.basename + str(1) + self.out_file.ext ]

    def startElement(self, name, attrs):
        XMLGenerator.startElement(self, name, attrs)
        self.context.append((name, attrs))
        
    def endElement(self, name):
        XMLGenerator.endElement(self, name)
        self.context.pop()
        if name == self.break_into:
            self.count += 1
            if self.count == self.break_after:
                self.count = 0
                for element in reversed(self.context):
                    self.out_file.write("\n".encode('utf-8'))
                    XMLGenerator.endElement(self, element[0])                   
                self.out_file.cycle()
                self.file_names.append( self.out_file.basename + str(self.out_file.index) + self.out_file.ext  )         
                XMLGenerator.startDocument(self)
                for element in self.context:
                    XMLGenerator.startElement(self, *element)
                    pd.DataFrame(self.file_names,columns=['columns']).to_csv("columns_iterator.csv",index=False)
def splitXML(file_name):
    filename, break_into, break_after = file_name,'node' ,50000
    parse(filename, XMLBreaker(break_into, int(break_after), out=CycleFile(filename)))

In [104]:
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import pandas as pd


def search(df,search_k_list,search_v_list,soup):
    takeaways = soup.findAll('node')
    for eachtakeaway in takeaways:
        name = ''
        another_tag = eachtakeaway('tag')
        for tag_attrs in another_tag:
            if str(tag_attrs['k'])=='name':
                name = str(tag_attrs['v'])
            elif str(tag_attrs['k']).lower() in search_k_list and str(tag_attrs['v']).lower() in search_v_list \
                and eachtakeaway['lon'] is not None and eachtakeaway['lat'] is not None:
                coordinates = [float(eachtakeaway['lon']), float(eachtakeaway['lat'])]
                df_new_record = pd.DataFrame([[str(tag_attrs['v']),name,None,coordinates]], columns=df.columns )
                df = df.append(df_new_record ,ignore_index=True)
    return df     


search_k_list = [
    'shop',
    'amenity',
    'office'
]

search_v_list = [
                'accountant','administrative','advertising_agency','architect','association',
                'bakery','bank','bar','beauty','biergarten','cafe','cinema','clothes','coffee','company',
                'convenience','dancing_school','dentist','doctors','driving_school','electronics',
                'engineering','estate_agent','fast_food','foundation','gambling','government',
                'greengrocer','hairdresser','insurance','internetservice','it','jewelry','kindergarten',
                'kiosk','labor_union','language_school','lawyer','mobile_phone','music_school','music',
                'musical_instrument','ngo','pharmacy','photo','photo_studio','physician','podologist',
                'political_party','pub','restaurant','school','selfmade_jewelry','shoes','supermarket',
                'tattoo','theatre','therapist','toys','travel_agent','university','variety_store',
                'video_games','video'
    ,'',' ',None,'post_office,''bicycle_rental','car_rental','community_centre','department_store','education_centre','fabric','laundry','butcher','alcohol'
                ]

df_consolidated = pd.DataFrame(columns=['Typ', 'Name','Zeiteinheiten','Adresse'])
df = pd.DataFrame(columns=['Typ', 'Name','Zeiteinheiten','Adresse'])

## OSM filename
splitXML('map.xml')

files = pd.read_csv("columns_iterator.csv")['columns']
#file = 'map.xml'
for file in files:
    handler = open(file).read()
    df_consolidated = df_consolidated.append(search(df,search_k_list,search_v_list,BeautifulSoup(handler,"lxml") ) )

    
df_consolidated.head(10)




,Typ,Name,Zeiteinheiten,Adresse
0,bank,,None,"[8.6904687, 50.8123311]"
1,restaurant,,None,"[8.6892236, 50.809141]"
2,convenience,Gutkauf,None,"[8.6892068, 50.8115642]"
3,pharmacy,,None,"[8.6903301, 50.8129661]"
4,bakery,Bäckerei Kraft,None,"[8.6903827, 50.8131183]"
5,dentist,,None,"[8.6904297, 50.8132477]"
6,doctors,,None,"[8.6894407, 50.8131465]"
7,cinema,,None,"[8.7734007, 50.8090021]"
8,cinema,,None,"[8.7737473, 50.8095579]"
9,bakery,Bäcker Müller,None,"[8.7781097, 50.8191913]"


In [105]:
df_consolidated.shape

(1101, 4)

135

,Typ,Name,Zeiteinheiten,Adresse
484,alcohol,Jaques' Wein Depot,None,"[8.7597673, 50.8008416]"
152,alcohol,Getränkemarkt Müller,None,"[8.6188325, 50.7421994]"
78,alcohol,Carl Kessler,None,"[8.7710015, 50.8093629]"
129,alcohol,Vinopolio Weinhandel und Vinothek Marburg,None,"[8.769164, 50.8049427]"


{'bakery',
 'beauty',
 'clothes',
 'coffee',
 'convenience',
 'electronics',
 'greengrocer',
 'hairdresser',
 'jewelry',
 'kiosk',
 'mobile_phone',
 'music',
 'musical_instrument',
 'photo',
 'photo_studio',
 'selfmade_jewelry',
 'shoes',
 'supermarket',
 'tattoo',
 'toys',
 'variety_store',
 'video',
 'video_games'}